In [ ]:
import pandapower as pp
import pandas as pd

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

sgen_table = pd.read_csv('sgen_table.csv')
sgen_table
# Install pandapower in a new environment by pip install pandapower==2.13.1

In [ ]:
# Assuming sgen_table contains your data with features X and target variable y
X = sgen_table.drop(columns=['target_column'])  # Features
y = sgen_table['target_column']  # Target variable

# 70% of the data for training
train_size = 0.7    

# Take the first 70% of all the data to train
n_train_samples = int(len(X) * train_size)

X_train, X_test = X[:n_train_samples], X[n_train_samples:]
y_train, y_test = y[:n_train_samples], y[n_train_samples:]

# Create an XGBRegressor model
model = XGBRegressor(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

In [ ]:
# Plot actual vs predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, predictions, color='blue', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='red', linewidth=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted Values')
plt.show()